In [1]:
# Imports
import os
from datetime import datetime
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
import re
import unicodedata
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ahocorasick import Automaton

from scipy.stats import zscore
from scipy.stats import mannwhitneyu

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer


# Settings
pd.set_option('display.max_rows', None)



Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Importing the Crunchbase Dataset to check the Number of NAs
cb_test = pd.read_csv('Documentation/Companies_Full_set.csv')
cb_test.info()

/var/folders/xt/0h_f_zjs1hvcdg6rdl1hs5_c0000gn/T/ipykernel_86218/1038015871.py:2: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  cb_test = pd.read_csv('Documentation/Companies_Full_set.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66311 entries, 0 to 66310
Data columns (total 22 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Unnamed: 0                              66311 non-null  int64  
 1   Organization Name                       66311 non-null  object 
 2   Founded Date                            66311 non-null  object 
 3   Website                                 65731 non-null  object 
 4   Headquarters Location                   65771 non-null  object 
 5   Industries                              65205 non-null  object 
 6   Number of Employees                     63392 non-null  object 
 7   CB Rank (Company)                       66311 non-null  object 
 8   Full Description                        63397 non-null  object 
 9   Industry Groups                         65205 non-null  object 
 10  Number of Funding Rounds                65334 non-null  fl

# Preparing Textual Data

## Importing results from first scrape

In [ ]:
# Preparing import paths for different batches
batches = [f"Batch_{i}" for i in range (1,11)]
years = range(2018, 2023)

file_paths = []

for batch in batches:
    for year in years:
        path = os.path.join('/Users/moritzwissel/Documents/GitHub/Master_Thesis_V3/Data/data_output/Batches', batch, f"website_info_{year}.csv")
        file_paths.append(path)


# Importing the different batches and years
dataframes = [] 
for path in file_paths:
    try:
        data_old = pd.read_csv(path)
        dataframes.append(data_old)  
    except FileNotFoundError:
        print(f'File not found: {path}')


# Combining imported dataframes
data_1 = pd.concat(dataframes, ignore_index=True)
data_1.to_excel('Data/scraped_websites_1.xlsx')
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7594 entries, 0 to 7593
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                7594 non-null   object 
 1   text_len               7594 non-null   int64  
 2   source                 7594 non-null   object 
 3   text                   7594 non-null   object 
 4   type                   7594 non-null   object 
 5   closest_snapshot       6723 non-null   object 
 6   closest_snapshot_time  6723 non-null   float64
 7   snapshot_in_window     7594 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 474.8+ KB


## Importing results from second scrape

In [ ]:
# Preparing import paths for different batches
batches = [f"Batch_{i}" for i in range (1,11)]
years = range(2018, 2023)

file_paths = []

for batch in batches:
    for year in years:
        path = os.path.join('/Users/moritzwissel/Documents/GitHub/Master_Thesis_V3/Data/data_output/Batches_V2', batch, f"website_info_{year}.csv")
        file_paths.append(path)


# Importing the different batches and years
dataframes = [] 
for path in file_paths:
    try:
        data_2 = pd.read_csv(path)
        dataframes.append(data_2)  
    except FileNotFoundError:
        print(f'File not found: {path}')


# Combining imported dataframes
data_2 = pd.concat(dataframes, ignore_index=True)
data_2.to_excel('Data/scraped_websites_2.xlsx')
data_2.info()

File not found: /Users/moritzwissel/Documents/GitHub/Master_Thesis_V3/Data/data_output/Batches_V2/Batch_8/website_info_2020.csv
File not found: /Users/moritzwissel/Documents/GitHub/Master_Thesis_V3/Data/data_output/Batches_V2/Batch_8/website_info_2021.csv
File not found: /Users/moritzwissel/Documents/GitHub/Master_Thesis_V3/Data/data_output/Batches_V2/Batch_8/website_info_2022.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7266 entries, 0 to 7265
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                7266 non-null   object 
 1   text_len               7266 non-null   int64  
 2   source                 7266 non-null   object 
 3   text                   7266 non-null   object 
 4   type                   7266 non-null   object 
 5   closest_snapshot       5786 non-null   object 
 6   closest_snapshot_time  5786 non-null   float64
 7   snapshot_in_window     7266 non-null 

## Importing Results from third scrape

In [ ]:
# Preparing import paths for different batches
batches = [f"Batch {i}" for i in range (1,11)]
years = range(2018, 2023)

file_paths = []

for batch in batches:
    for year in years:
        path = os.path.join('/Users/moritzwissel/Documents/GitHub/Master_Thesis_V3/Data/data_output/Batches_V3', batch, f"website_info_{year}.csv")
        file_paths.append(path)


# Importing the different batches and years
dataframes = [] 
for path in file_paths:
    try:
        data_3 = pd.read_csv(path)
        dataframes.append(data_3)  
    except FileNotFoundError:
        print(f'File not found: {path}')


# Combining imported dataframes
data_3 = pd.concat(dataframes, ignore_index=True)
data_3.to_excel('Data/scraped_websites_3.xlsx')
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8053 entries, 0 to 8052
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                8053 non-null   object 
 1   text_len               8053 non-null   int64  
 2   source                 8053 non-null   object 
 3   text                   8053 non-null   object 
 4   type                   8053 non-null   object 
 5   closest_snapshot       6184 non-null   object 
 6   closest_snapshot_time  6184 non-null   float64
 7   snapshot_in_window     8053 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 503.4+ KB


## Importing results from AI scrape

In [ ]:
# Preparing import paths for different batches
batches = [f"Batch {i}" for i in range (1,2)]
years = range(2018, 2023)

file_paths = []

for batch in batches:
    for year in years:
        path = os.path.join('/Users/moritzwissel/Documents/GitHub/Master_Thesis_V3/Data/data_output/Batches_AI', batch, f"website_info_{year}.csv")
        file_paths.append(path)


# Importing the different batches and years
dataframes = [] 
for path in file_paths:
    try:
        data_AI = pd.read_csv(path)
        dataframes.append(data_AI)  
    except FileNotFoundError:
        print(f'File not found: {path}')


# Combining imported dataframes
data_AI = pd.concat(dataframes, ignore_index=True)
data_AI.to_excel('Data/scraped_websites_AI.xlsx')
data_AI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1206 entries, 0 to 1205
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                1206 non-null   object 
 1   text_len               1206 non-null   int64  
 2   source                 1206 non-null   object 
 3   text                   1206 non-null   object 
 4   type                   1206 non-null   object 
 5   closest_snapshot       1043 non-null   object 
 6   closest_snapshot_time  1043 non-null   float64
 7   snapshot_in_window     1206 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 75.5+ KB


## Completing the new Sample

To make the subsequents scrapings more efficient, I checked which companies form the previous scrapings are also in the new sample, and only scraped the difference. Now I have to complete the second scraping results based on the new random sample with the overlapping companies from the previous sample.

In [ ]:
# Importing the data
overlap_1 = pd.read_csv('Data/Crunchbase/overlapping_websites.csv')
overlap_2 = pd.read_csv('Data/Crunchbase/overlapping_websites_S3.csv')
overlap_AI = pd.read_csv('Data/Crunchbase/overlapping_websites_AI.csv')

In [ ]:
# Completing scrape 2 with overlapping previously scraped websites
websites_in_overlap_1 = overlap_1['website'].unique()
data_1_subset = data_1[data_1['website'].isin(websites_in_overlap_1)]

data_combined_2 = pd.concat([data_2, data_1_subset])
data_combined_2.drop_duplicates(subset=['website'], keep='first', inplace=True)
data = data_combined_2


In [ ]:
# Completing scrape 3 with overlapping previously scraped websites
websites_in_overlap_2 = overlap_2['website'].unique()
data_2_subset = data[data['website'].isin(websites_in_overlap_2)]

data_combined_3 = pd.concat([data_3, data_2_subset])
data_combined_3.drop_duplicates(subset=['website'], keep='first', inplace=True)
data = data_combined_3


In [ ]:
# Completing AI Scrape with overlapping previously scraped websites
websites_in_overlap_AI = overlap_AI['website'].unique()
data_AI_subset = data[data['website'].isin(websites_in_overlap_AI)]

data_combined_AI = pd.concat([data_AI, data_AI_subset])
data_combined_AI.drop_duplicates(subset=['website'], keep='first', inplace=True)
data_AI = data_combined_AI


# Adding the AI scraping as to the main data
data = pd.concat([data, data_AI])
data.drop_duplicates(subset=['website'], keep='first', inplace=True)
data.to_excel('Data/final_combined_data.xlsx')
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 10701 entries, 0 to 1205
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                10701 non-null  object 
 1   text_len               10701 non-null  int64  
 2   source                 10701 non-null  object 
 3   text                   10701 non-null  object 
 4   type                   10701 non-null  object 
 5   closest_snapshot       8381 non-null   object 
 6   closest_snapshot_time  8381 non-null   float64
 7   snapshot_in_window     10701 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 752.4+ KB


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10701 entries, 0 to 1205
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                10701 non-null  object 
 1   text_len               10701 non-null  int64  
 2   source                 10701 non-null  object 
 3   text                   10701 non-null  object 
 4   type                   10701 non-null  object 
 5   closest_snapshot       8381 non-null   object 
 6   closest_snapshot_time  8381 non-null   float64
 7   snapshot_in_window     10701 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 752.4+ KB


## Pre-Processing Text

## Selecting Valid Websites
First, I use a code excerpt of Guzman & Li (2023) to exclude websites that have likely invalid text.

In [ ]:
# 
invalid_conditions = [
    data['text'].str.contains('BuyDomains.com'),
    data['text'].str.contains('This Web page is parked for FREE'),
    data['text'].str.contains('Free business profile for provided by Network Solutions'),
    data['text'].str.contains('The Sponsored Listings displayed above are served automatically'),
    data['text'].str.contains('Apache'),
    data['text'].str.contains('website is for sale'), 
    data['text'].str.contains('This Web site coming soon'),
    data['text'].str.contains('Welcome to the new website! Our site has been recently created'),
    data['text'].str.contains('Wayback Machine'),
    data['text'].str.contains('Wayback Machine See what s new with book lending'),
    data['text'].str.contains('AVAILABLE NOT FOUND'),
    data['text'].str.contains('^DefaultHomePage'),
    data['text'].str.contains('^I?n?t?ernet Archive: Scheduled Mantenance'),
    data['text'].str.contains('The page cannot be found'),
    data['text'].str.contains('503'),
    data['text'].str.contains('^5?0?3 Service Unavailable'),
    data['text'].str.lower().str.contains('domain down'),
    data['text'].str.contains('^Too Many Requests'),
    data['text'].str.contains('^Your browser does not support'),
    data['text'].str.contains('^New Server for COMPANYNAME'),
    data['text'].str.contains('this page is parked FREE'),
    data['text'].str.contains('domain name was recently registered'),
    data['text'].str.contains('placeholder for domain'),
    data['text'].str.contains('xtremedata.com  : Low cost'),
    data['text'].str.lower().str.contains('domain name registration'),
    data['text'].str.contains('Under Construction'),
    data['text'].str.contains('This Web site is currently under'),
    data['text'].str.contains('This domain name was recently'),
    data['text'].str.contains('This page is parked free'),
    data['text'].str.contains('^Microsoft VBScript runtime error'),
    data['text'].str.contains('^WebFusion'),
    data['text'].str.contains('^Register domain names'),
    data['text'].str.contains('^Moved This page has moved'),
    data['text'].str.contains('^Coming Soon'),
    data['text'].str.contains('Site (is )?Temporarily Unavailable'),
    data['text'].str.contains('^Under Construction'),
    data['text'].str.contains('^cPanel'),
    data['text'].str.contains('^Authorization Required'),
    data['text'].str.contains('^Top Web Search Directory Top Web Searches'),
    data['text'].str.contains('^Web Searches'),
    data['text'].str.contains('^Web Hosting'),
    data['text'].str.contains('^Search Directory Page Sponsored Listing'),
    data['text'].str.contains('^coming soon'),
    data['text'].str.contains('This site is the default web server site'),
    data['text'].str.contains('DF-1.4 %���� 0 obj< '),
    data['text'].str.contains('This page uses frames, but your brow'),
    data['text'].str.contains('U N D E R C O N S T R U C T I O N'),
    data['text'].str.contains('We recommend you upgrade your browser to one of below free alternatives'),
    data['text'].str.contains('For full functionality of this site it is necessary to enable Here are the instructions how to enable JavaScript in your web browser.'),
    data['text'].str.lower().str.contains('under construction'),
    data['text'].str.contains('Page cannot be Please contact your service provider for more'),
    data['text'].str.contains('^A WordPress Site'),
    data['text'].str.contains('^Related Searches: Related Searches'),
    data['text'].str.contains('^Welcome to IIS'), 
    data['text'].str.contains('^crypto'),
    data['text'].str.contains('^web3'),
    data['text'].str.contains('^blockchain'),
    data['text'].str.contains('^domain')
]

# Combining all conditions
invalid_mask = invalid_conditions[0]
for condition in invalid_conditions[1:]:
    invalid_mask = invalid_mask | condition

# Filtering out the invalid websites
data = data[~invalid_mask]


/var/folders/xt/0h_f_zjs1hvcdg6rdl1hs5_c0000gn/T/ipykernel_86218/847701268.py:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['text'].str.contains('Site (is )?Temporarily Unavailable'),


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8367 entries, 0 to 1204
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                8367 non-null   object 
 1   text_len               8367 non-null   int64  
 2   source                 8367 non-null   object 
 3   text                   8367 non-null   object 
 4   type                   8367 non-null   object 
 5   closest_snapshot       6596 non-null   object 
 6   closest_snapshot_time  6596 non-null   float64
 7   snapshot_in_window     8367 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 588.3+ KB


## Normalizing Data

In [ ]:
# Defining pre-processing functions 
def normalize_text(text):
    """
    Normalizes text by removing special characters and transforming everything to lowercase

    Parameters: 
    - text: loaded up text for each row of the data scraping
    """
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def tokenize_and_process(text):
    """
    Tokenizes the text and removes stopwords.

    Parameters: 
    - text: loaded up text for each row of the data scraping
    """
    # Tokenizing text
    tokens = word_tokenize(text)
    
    # Removing stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Not applying stemming as that significantly degrades the matching algorithms
    return filtered_tokens



In [ ]:
# Normalizing and preprocessing the text
data['text_normalized'] = data['text'].apply(normalize_text)
data['tokens'] = data['text_normalized'].apply(tokenize_and_process)

# Converting token lists back to strings for further analysis
data['text_preprocessed'] = data['tokens'].apply(lambda tokens: ' '.join(tokens))

# Dropping intermediate columns as they are no longer needed
data.drop(['text_normalized', 'tokens'], axis=1, inplace=True)

# Saving the dataframe for faster workflows
data.to_excel('Exports/data_normalized.xlsx')

# Applying the Keywords

As there are multiple lists of possible keywords, I'm going to check them and their combinations to find the one that is suited best for this task.

In [ ]:
# Importing the different lists of keywords
data_long = data.copy()         # Full Set of keywords from all keyword sources
data_combined = data.copy()    
data_nakazato = data.copy()     # Nakazato Keywords
data_final = data.copy()        # Final selection of keywords used for following analyses

df_keywords = pd.read_excel('Data/Keywords/keywords_baruffaldi.xlsx')
df_keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   keyword  206 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


## Creating Aho-Corasick Search Algorithm

In [ ]:
def build_automaton_and_search(text_data, keywords):
    """
    Builds an Aho-Corasick automaton with the given keywords and applies it to the given text data.

    Parameters:
    - text_data: A pandas Series containing text to search through.
    - keywords: A set of keywords to build the automaton with.

    Returns:
    - A DataFrame with columns for keyword count and matched keywords for each text entry.
    """
    # Initializing the automaton
    A = Automaton()
    for idx, key in enumerate(keywords):
        A.add_word(key, (idx, key))
    A.make_automaton()

    # Defining the search function
    def search(text):
        count = 0
        matched_keywords = set()
        for end_index, (idx, original_value) in A.iter(text.lower()):
            count += 1
            matched_keywords.add(original_value.strip())
        return count, ', '.join(matched_keywords)

    # Applying the search function to the text data
    result = text_data.apply(search)
    
    # Spliting the result into two separate columns for return
    keyword_count = result.apply(lambda x: x[0])
    matched_keywords = result.apply(lambda x: x[1])
    
    return pd.DataFrame({'ai_keyword_count': keyword_count, 'matched_keywords': matched_keywords})



## 1. Baruffaldi Keywords
Using the previously crafted list of ai-related keywords and acronyms, comprising the Nakaza

In [ ]:
# Using the search function with the first set of keywords
ai_keywords = set(df_keywords['keyword'].str.lower())

results = build_automaton_and_search(data['text'], ai_keywords)
data['ai_keyword_count'] = results['ai_keyword_count']
data['matched_keywords'] = results['matched_keywords']

# Checking the dataframe structure
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 8367 entries, 0 to 1204
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                8367 non-null   object 
 1   text_len               8367 non-null   int64  
 2   source                 8367 non-null   object 
 3   text                   8367 non-null   object 
 4   type                   8367 non-null   object 
 5   closest_snapshot       6596 non-null   object 
 6   closest_snapshot_time  6596 non-null   float64
 7   snapshot_in_window     8367 non-null   float64
 8   text_preprocessed      8367 non-null   object 
 9   ai_keyword_count       8367 non-null   int64  
 10  matched_keywords       8367 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 784.4+ KB


In [ ]:
# Checking the number of keywords found and their frequency
keyword_count_frequency = data['ai_keyword_count'].value_counts().sort_index()
keyword_frequency = data['matched_keywords'].value_counts().sort_index()

# Displaying the data
display(keyword_count_frequency)
display(keyword_frequency)

ai_keyword_count
0     7557
1      305
2      168
3       88
4       73
5       33
6       40
7       20
8       15
9       13
10       8
11       8
12       4
13       4
14       2
15       5
16       2
17       1
18       3
20       1
21       2
22       1
24       1
25       1
27       2
28       1
30       1
33       1
35       1
41       1
42       1
43       1
45       1
49       1
55       1
Name: count, dtype: int64

matched_keywords
                                                                                                                                                                                            7557
artificial neural network, decision tree, machine learning, neural network                                                                                                                     1
artificial neural network, machine learning, robot, neural network                                                                                                                             1
artificial neural network, neural network                                                                                                                                                      3
autonomous vehicle                                                                                                                                                                             5
autonomous vehicle

In [ ]:
# Adding a binary indicator based on the keyword counter
def keyword_indicator(count):
    return 1 if count > 0 else 0

data['keyword_bin'] = data['ai_keyword_count'].apply(keyword_indicator)
print(data['keyword_bin'].sum())

810


## 2. Full Keywords
Using the long list built from all papers that published lists of keywords to identify the use of AI in textual data

In [ ]:
# Importing the manually created long-list
df_keywords_long = pd.read_excel('/Users/moritzwissel/Library/CloudStorage/OneDrive-Personal/Master/Master Thesis/Working Documents/Resource Overviews Thesis.xlsx', sheet_name= 'AI-Keywords')
df_keywords_long = df_keywords_long.drop(columns= ['author', 'year'])
ai_keywords_long = set(' ' + kw.lower() + ' ' for kw in df_keywords_long['keyword'])

# Applying the search algorithm with the new data
results_long = build_automaton_and_search(data_long['text'], ai_keywords_long)
data_long['ai_keyword_count'] = results_long['ai_keyword_count']
data_long['matched_keywords'] = results_long['matched_keywords']

# Checking the dataframe structure
data_long.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8367 entries, 0 to 1204
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                8367 non-null   object 
 1   text_len               8367 non-null   int64  
 2   source                 8367 non-null   object 
 3   text                   8367 non-null   object 
 4   type                   8367 non-null   object 
 5   closest_snapshot       6596 non-null   object 
 6   closest_snapshot_time  6596 non-null   float64
 7   snapshot_in_window     8367 non-null   float64
 8   text_preprocessed      8367 non-null   object 
 9   ai_keyword_count       8367 non-null   int64  
 10  matched_keywords       8367 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 784.4+ KB


In [ ]:
# Checking the number of keywords found and their frequency
keyword_count_frequency_long = data_long['ai_keyword_count'].value_counts().sort_index()
keyword_frequency_long = data_long['matched_keywords'].value_counts().sort_index()

# Displaying the data 
display(keyword_count_frequency_long)
display(keyword_frequency_long)

ai_keyword_count
0     2989
1     1293
2      901
3      578
4      496
5      328
6      269
7      204
8      189
9      159
10     114
11     101
12      83
13      74
14      64
15      45
16      45
17      37
18      33
19      31
20      37
21      30
22      28
23      12
24      21
25      16
26      20
27      18
28      10
29      10
30      12
31       8
32       9
33      10
34       7
35       5
36       8
37       5
38       1
39       6
40       6
41       4
42       4
43       5
44       4
45       1
46       3
47       5
48       2
49       3
51       1
52       3
53       1
55       3
56       1
57       1
58       1
59       3
60       1
61       1
65       1
70       2
77       1
84       1
86       1
91       1
92       1
Name: count, dtype: int64

matched_keywords
                                                                                                                                                                                                                                                                                                                                                                                  2989
active learning, google, deep learning, logistic regression, ml, embeddings, neural networks                                                                                                                                                                                                                                                                                         1
active learning, machine learning models, forecasting, autonomous, ai, learning models, learning algorithm, algorithm, deep learning, ml, machine learning, autonomous vehicle                                                           

In [ ]:
# Adding a binary indicator based on the keyword counter
def keyword_indicator(count):
    return 1 if count > 0 else 0

data_long['keyword_bin'] = data_long['ai_keyword_count'].apply(keyword_indicator)
print(data_long['keyword_bin'].sum())

5378


## 3. Nakazato & Squicciarini
This search only includes the improved list of AI-related keywords of Nakazato & Squicciarini

In [ ]:
# Imporitng the new list of keywords
df_keywords_combined = pd.read_excel('Data/Keywords/keywords_nakazato.xlsx')

In [ ]:
# Adding Acronyms of the used keywords
df_acronyms_combined = pd.read_excel('Data/Keywords/keywords_nakazato_Abbreviations.xlsx')

# Correctly combining the two dataframes
keywords_full = pd.concat([df_keywords_combined, df_acronyms_combined], ignore_index=True)
keywords_full = set(' ' + kw.lower() + ' ' for kw in keywords_full['keyword'])

# Creating a set of keywords including both keywords and acronyms, with spaces added
keywords_nakazato =  set(' ' + kw.lower() + ' ' for kw in df_keywords_combined['keyword'])

print(f"Number of keywords (incl. acronyms{len(keywords_nakazato)}")

# Applying the search algorithm to the data
results_nakazato = build_automaton_and_search(data_nakazato['text'], keywords_nakazato)
data_nakazato['ai_keyword_count'] = results_nakazato['ai_keyword_count']
data_nakazato['matched_keywords'] = results_nakazato['matched_keywords']

# Checking the dataframe strucutre
data_nakazato.info()

Number of keywords (incl. acronyms105
<class 'pandas.core.frame.DataFrame'>
Index: 8367 entries, 0 to 1204
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                8367 non-null   object 
 1   text_len               8367 non-null   int64  
 2   source                 8367 non-null   object 
 3   text                   8367 non-null   object 
 4   type                   8367 non-null   object 
 5   closest_snapshot       6596 non-null   object 
 6   closest_snapshot_time  6596 non-null   float64
 7   snapshot_in_window     8367 non-null   float64
 8   text_preprocessed      8367 non-null   object 
 9   ai_keyword_count       8367 non-null   int64  
 10  matched_keywords       8367 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 784.4+ KB


In [ ]:
# Checking the number of keywords found and their frequency
keyword_count_frequency_nakazato = data_nakazato['ai_keyword_count'].value_counts().sort_index()
keyword_frequency_nakazato = data_nakazato['matched_keywords'].value_counts().sort_index()

# Displaying the results
display(keyword_count_frequency_nakazato)
display(keyword_frequency_nakazato)

ai_keyword_count
0     7690
1      292
2      154
3       77
4       48
5       26
6       29
7        7
8        9
9        5
10       9
11       4
12       1
13       2
14       4
15       4
16       1
19       3
21       1
46       1
Name: count, dtype: int64

matched_keywords
                                                                                                                                            7690
artificial intelligence                                                                                                                      178
artificial intelligence, chatbot                                                                                                               2
artificial intelligence, data mining, natural language processing                                                                              1
artificial intelligence, deep learning                                                                                                        10
artificial intelligence, deep learning, autoencoder, backpropagation, neural network                                                           1
artificial intelligence, deep learning, neural network, machine learning, chatbot                                

## 4. Comparing results

In [ ]:
# Matching the keyword_count_frequency dataframes
matched_results = pd.concat([keyword_count_frequency, keyword_count_frequency_long, keyword_count_frequency_nakazato], axis=1)
matched_results.reset_index(inplace=True)
matched_results.columns = ['Keyword Hits','Frequency', 'Frequency_Long', 'Frequency_Nakazato']
matched_results.index.name = 'Keyword'


# Displaying the matched results
display(matched_results)

,Keyword Hits,Frequency,Frequency_Long,Frequency_Nakazato
Keyword,,,,
0,0,7557.0,2989,7690.0
1,1,305.0,1293,292.0
2,2,168.0,901,154.0
3,3,88.0,578,77.0
4,4,73.0,496,48.0
5,5,33.0,328,26.0
6,6,40.0,269,29.0
7,7,20.0,204,7.0
8,8,15.0,189,9.0


In [ ]:
# Creating a function to calculate percentages and absolute numbers of keywords in the data
def calculate_keyword_distribution(df):
    total_entries = len(df)
    zero_hits = len(df[df['ai_keyword_count'] == 0])
    above_zero_hits = total_entries - zero_hits
    return {
        "No Hits (%)": zero_hits / total_entries * 100,
        "No Hits (#)": zero_hits,
        ">0 Hits (%)": above_zero_hits / total_entries * 100,
        ">0 Hits (#)": above_zero_hits
    }

# Applying the function to each dataframe
distribution_data = {
    "Baruffaldi": calculate_keyword_distribution(data),
    "All Keywords": calculate_keyword_distribution(data_long),
    "Nakazato & Squicciarini": calculate_keyword_distribution(data_nakazato)
}

# Creating a dataframe from the aggregated data
aggregated_df = pd.DataFrame(distribution_data).transpose()

# Displaying the dataframe
display(aggregated_df)


,No Hits (%),No Hits (#),>0 Hits (%),>0 Hits (#)
Baruffaldi,90.319111,7557.0,9.680889,810.0
All Keywords,35.723676,2989.0,64.276324,5378.0
Nakazato & Squicciarini,91.908689,7690.0,8.091311,677.0


# Iterating & Optimizing the Keywords
After the first round of testing with the keyword set that also included the acronyms, I noticed that the amount of Positives increased strongly. Therefore, I grew suspicious and manually checked all website-texts that were found to have acronyms in it. With that I wanted to assess if these positives are false positives or true positives.

In a first high-level check, I found obvious acronyms that lead to fals positives: USPS state acronyms (e.g. "FL" for "Florida" but also "Feature Learning")
Thus, I removed all acronyms that are also state postal acronyms.

In [ ]:
# Importing US states acronyms
us_states_df = pd.read_excel('Data/Keywords/US_State_Acronyms.xlsx')
us_states_set = set(' ' + kw.lower() + ' ' for kw in us_states_df['Acronym'])
print(us_states_set)

{' ut ', ' az ', ' id ', ' la ', ' va ', ' ma ', ' wa ', ' ar ', ' hi ', ' oh ', ' sd ', ' tx ', ' ga ', ' co ', ' nm ', ' mi ', ' ok ', ' pa ', ' wy ', ' ky ', ' nc ', ' ks ', ' nv ', ' md ', ' wv ', ' ms ', ' in ', ' mn ', ' nd ', ' ny ', ' fl ', ' me ', ' sc ', ' wi ', ' ak ', ' mo ', ' vt ', ' tn ', ' de ', ' nh ', ' ca ', ' nj ', ' mt ', ' ne ', ' ri ', ' or ', ' ia ', ' ct ', ' al ', ' il '}


In [ ]:
# Adding the List of AI-related Acronyms
print(df_keywords_combined)
print("AND")
print(df_acronyms_combined)

keywords_full = pd.concat([df_keywords_combined, df_acronyms_combined], ignore_index=True)
keywords_full_v2 = set(' ' + kw.lower() + ' ' for kw in keywords_full['keyword'])


                                    keyword
0                                  Adaboost
1                       Adversarial network
2                                Ant colony
3                   Artificial intelligence
4                          Association rule
5                               Autoencoder
6                       Autonomic computing
7                           Backpropagation
8                          Bayesian network
9                                Bee colony
10                                  Chatbot
11                      Cognitive computing
12                  Collaborative filtering
13               Computational intelligence
14                            Connectionism
15                            Connectionist
16                              Data mining
17                           Decision model
18                            Decision tree
19                      Deep belief network
20                            Deep learning
21         Differential evolutio

### Manual Check of False positives
Next, I manually check all website-texts where acronyms only appeared by themselves, without any other "written-out" keywords, to assess whether the acronyms appeared with their AI-related meaning (True Positive, i.e. "TP"), or in another irrelevant context (False Positives, i.e. "FP"). 

In [ ]:
print(keywords_full_v2)
print(len(keywords_full_v2))

{' sentiment analysis ', ' backpropagation ', ' generative adversarial network ', ' natural gradient ', ' lda ', ' sparse coding ', ' pattern recognition ', ' pgm ', ' ma ', ' high-dimensional data ', ' autoencoder ', ' ga ', ' hidden markov model ', ' artificial intelligence ', ' feature extraction ', ' fs ', ' q-learning ', ' mlc ', ' memetic algorithm ', ' object recognition ', ' gmm ', ' support vector machine ', ' high-dimensional input ', ' gb ', ' fl ', ' multi-agent system ', ' ai ', ' gaussian mixture model ', ' vector machine ', ' sr ', ' collaborative filtering ', ' bee colony ', ' association rule ', ' neuromorphic computing ', ' or ', ' lr ', ' mas ', ' aco ', ' pso ', ' latent semantic analysis ', ' lm ', ' multi-objective optimisation ', ' dimensionality reduction ', ' logistic regression ', ' swarm behavior ', ' xgb ', ' ea ', ' machine intelligence ', ' dl ', ' high-dimensional model ', ' neural network ', ' computational intelligence ', ' natural language generation '

In [ ]:
# Applying the new keywords to check for false positives
data_nakazato_acr = data_nakazato.copy()
print(keywords_full)
results_nakazato_V2 = build_automaton_and_search(data_nakazato_acr['text'], keywords_full_v2)
data_nakazato_acr['ai_keyword_count'] = results_nakazato_V2['ai_keyword_count']
data_nakazato_acr['matched_keywords'] = results_nakazato_V2['matched_keywords']

# Check the dataframe structure
data_nakazato_acr.info()

                                    keyword
0                                  Adaboost
1                       Adversarial network
2                                Ant colony
3                   Artificial intelligence
4                          Association rule
5                               Autoencoder
6                       Autonomic computing
7                           Backpropagation
8                          Bayesian network
9                                Bee colony
10                                  Chatbot
11                      Cognitive computing
12                  Collaborative filtering
13               Computational intelligence
14                            Connectionism
15                            Connectionist
16                              Data mining
17                           Decision model
18                            Decision tree
19                      Deep belief network
20                            Deep learning
21         Differential evolutio

In [ ]:
# Checking the number of keywords found and their frequency
keyword_count_frequency_nakazato_acronyms = data_nakazato_acr['ai_keyword_count'].value_counts().sort_index()
keyword_frequency_nakazato_acronyms = data_nakazato_acr['matched_keywords'].value_counts().sort_index()

# Displaying the findings
print(keyword_count_frequency_nakazato_acronyms)
print(keyword_frequency_nakazato_acronyms)

ai_keyword_count
0      2012
1      1208
2      1017
3       793
4       662
5       492
6       404
7       306
8       246
9       175
10      133
11      112
12      108
13       82
14       74
15       63
16       50
17       50
18       44
19       31
20       25
21       23
22       20
23       20
24       19
25       16
26       18
27       17
28       15
29       17
30       19
31       10
32        6
33        9
34        9
35        5
36        6
37        4
38        1
39        5
40        3
41        5
42        4
43        6
44        6
45        1
46        3
48        2
53        1
54        1
56        1
61        1
63        1
68        1
70        2
71        1
73        1
202       1
Name: count, dtype: int64
matched_keywords
                                                                                                                                                             2012
ai                                                                                

In [ ]:
# Calculating the keyword distribution for the new dataframe
new_data_distribution = calculate_keyword_distribution(data_nakazato_acr)

# Appending the new data to the existing dictionary
distribution_data["Nakazato & Squicciarini + Acronyms"] = new_data_distribution

# Re-creating the aggregated dataframe
aggregated_df = pd.DataFrame(distribution_data).transpose()

display(aggregated_df)


,No Hits (%),No Hits (#),>0 Hits (%),>0 Hits (#)
Baruffaldi,90.319111,7557.0,9.680889,810.0
All Keywords,35.723676,2989.0,64.276324,5378.0
Nakazato & Squicciarini,91.908689,7690.0,8.091311,677.0
Nakazato & Squicciarini + Acronyms,24.046851,2012.0,75.953149,6355.0


Looking at the results, the jump in found keywords form the base dataset of Nakazator & Squicciarini to one extended with acronyms is striking. 
This is why in a next step, I will manually check all acronyms to determine the Precision. 
I chose precision as the decision criteria, because of the classification for further analysis, it is more important to minimize the number of false positives, as they could skew the analysis heavily. On the other hand with a high number of classified websites, missing some websites is less detrimental to the validity of my research.

CITED CHOUDHURY ET AL.

### Potential Graveyard Export of Manual Check

In [ ]:
expanded_keywords = data_nakazato.explode('matched_keywords')

# Counting unique occurences
keyword_frequency_nakazato = expanded_keywords.groupby(['text', 'matched_keywords']).size().reset_index(name='counts')
keyword_frequency_nakazato.to_excel("./Documentation/keyword_count.xlsx", index=False)

# Counting unique keywords across website texts
keyword_frequency_unique = expanded_keywords['matched_keywords'].value_counts().reset_index()
keyword_frequency_unique.columns = ['matched_keywords', 'counts']  # Renaming columns for clarity

# Exporting unique keyword counts to Excel
keyword_frequency_unique.to_excel("./Documentation/keyword_count_unique.xlsx", index=False)


In [ ]:
# Creating a boolean mask for filtering
mask = keyword_frequency_nakazato['matched_keywords'].isin(df_acronyms_combined)

# Applying the mask to filter rows where 'matched_keywords_combined' is in 'df_acronyms_combined' set
filtered_keyword_frequency_combined = keyword_frequency_nakazato[mask]

# Export the filtered DataFrame to Excel
filtered_keyword_frequency_combined.to_excel("./Documentation/keyword_count_filtered.xlsx", index=False)


In [ ]:
# Adding a binary indicator based on the keyword counter
def keyword_indicator(count):
    return 1 if count > 0 else 0

data_combined['keyword_bin'] = data_nakazato['ai_keyword_count'].apply(keyword_indicator)
print(data_combined['keyword_bin'].sum())

677


### Analysing results

In [ ]:
# Analysing the results of the manual check
# Importing the file
manual_check = pd.read_excel('Documentation/keyword_count_manual_Important.xlsx', sheet_name= 'All checked')
manual_check.columns = ['Website_text', 'Keyword', 'Amount', 'Verdict']

# Calculating the percentage of true positives (TP) and converting the verdict into a binary classification
manual_check['Verdict'] = manual_check['Verdict'].apply(lambda x: 1 if x == 'TP' else 0)

# Grouping the data by keyword (i.e. acronym) and calculating the sum of TP and the total entries
check_results = manual_check.groupby('Keyword').agg(
    TP_count = ('Verdict', 'sum'),
    Total_count = ('Verdict', 'count')
)

# Creating a results table
check_results['Precision'] = (check_results['TP_count']/ check_results['Total_count'])
check_results = check_results.sort_values(['Precision'], ascending = False)
check_results.reset_index(inplace = True)

display(check_results)
print(check_results.info())

,Keyword,TP_count,Total_count,Precision
0,ai,343,350,0.980000
1,ml,21,25,0.840000
2,nlp,2,3,0.666667
3,dr,1,32,0.031250
4,pr,1,42,0.023810
5,si,0,13,0.000000
6,mas,0,3,0.000000
7,rf,0,10,0.000000
8,sa,0,5,0.000000
9,aco,0,2,0.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Keyword      32 non-null     object 
 1   TP_count     32 non-null     int64  
 2   Total_count  32 non-null     int64  
 3   Precision    32 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.1+ KB
None


As previously stated, I chose to opt for a high precision in the keywords, and thus selected only Acronyms with a precision greater than 0.8
The results consequently show that only 2 ('AI' and 'ML') out of the 32 proposed acronyms are suitable to detect the mentioning of AI in website texts.

Therefore all other keywords with insufficient true positives will get removed from the list of acronyms.

In [ ]:
# Selecting only the relevant acronyms with a TP rate of over 80%
relevant_results = check_results[check_results['Precision']>0.8]
suitable_acronyms = set(' ' + kw.lower() + ' ' for kw in relevant_results['Keyword'])

# Since only the acronyms 'AI' and 'ML' remain, I will simply add them to the list of keywords
print(keywords_nakazato)
print(suitable_acronyms)

keywords_final = keywords_nakazato | suitable_acronyms


print(f"Refined keyword set size: {len(keywords_final)}")
display(keywords_final)


{' sentiment analysis ', ' backpropagation ', ' generative adversarial network ', ' natural gradient ', ' sparse coding ', ' pattern recognition ', ' high-dimensional data ', ' autoencoder ', ' hidden markov model ', ' artificial intelligence ', ' feature extraction ', ' q-learning ', ' memetic algorithm ', ' object recognition ', ' support vector machine ', ' high-dimensional input ', ' multi-agent system ', ' gaussian mixture model ', ' vector machine ', ' collaborative filtering ', ' bee colony ', ' association rule ', ' neuromorphic computing ', ' latent semantic analysis ', ' multi-objective optimisation ', ' dimensionality reduction ', ' logistic regression ', ' swarm behavior ', ' machine intelligence ', ' high-dimensional model ', ' neural network ', ' computational intelligence ', ' natural language generation ', ' feature selection ', ' kernel learning ', ' fuzzy logic ', ' learning algorithm ', ' semi-supervised learning ', ' bayesian network ', ' deep learning ', ' long sho

{' adaboost ',
 ' adversarial network ',
 ' ai ',
 ' ant colony ',
 ' artificial intelligence ',
 ' association rule ',
 ' autoencoder ',
 ' autonomic computing ',
 ' backpropagation ',
 ' bayesian network ',
 ' bee colony ',
 ' chatbot ',
 ' cognitive computing ',
 ' collaborative filtering ',
 ' computational intelligence ',
 ' connectionism ',
 ' connectionist ',
 ' data mining ',
 ' decision model ',
 ' decision tree ',
 ' deep belief network ',
 ' deep learning ',
 ' differential evolution algorithm ',
 ' dimensionality reduction ',
 ' ensemble learning ',
 ' evolutionary algorithm ',
 ' evolutionary computation ',
 ' expert system ',
 ' factorisation machine ',
 ' feature engineering ',
 ' feature extraction ',
 ' feature learning ',
 ' feature selection ',
 ' fuzzy algorithm ',
 ' fuzzy c ',
 ' fuzzy environment ',
 ' fuzzy logic ',
 ' fuzzy number ',
 ' fuzzy set ',
 ' fuzzy system ',
 ' gaussian mixture model ',
 ' gaussian process ',
 ' generative adversarial network ',
 ' ge

# Iterating and Optimizing the search algorithm
After creating a suitable list of keywords for the search algorithm, I will next optimize the search algorithm through iterative improvement

## Aho-Corasick Search Algorithm

In [ ]:
# Running the final keywords with the Aho-Corasick search algorithm
results_aho = build_automaton_and_search(data_final['text'], keywords_final)
data_final['ai_keyword_count'] = results_aho['ai_keyword_count']
data_final['matched_keywords'] = results_aho['matched_keywords']

# Checking the dataframe structure
data_nakazato.info()

# Checking the number of keywords found and their frequency
keyword_count_frequency_aho = data_final['ai_keyword_count'].value_counts().sort_index()
keyword_count_frequency_aho = data_final['matched_keywords'].value_counts().sort_index()

# Displaying the results
print(keyword_count_frequency_aho)
print(keyword_count_frequency_aho)

<class 'pandas.core.frame.DataFrame'>
Index: 8367 entries, 0 to 1204
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                8367 non-null   object 
 1   text_len               8367 non-null   int64  
 2   source                 8367 non-null   object 
 3   text                   8367 non-null   object 
 4   type                   8367 non-null   object 
 5   closest_snapshot       6596 non-null   object 
 6   closest_snapshot_time  6596 non-null   float64
 7   snapshot_in_window     8367 non-null   float64
 8   text_preprocessed      8367 non-null   object 
 9   ai_keyword_count       8367 non-null   int64  
 10  matched_keywords       8367 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 784.4+ KB
matched_keywords
                                                                                                                                             

In [ ]:
# Adding the results to the Overview
aho_data_distribution = calculate_keyword_distribution(data_final)
distribution_data['Keywords Final'] = aho_data_distribution
aggregated_df = pd.DataFrame(distribution_data).transpose()

display(aggregated_df)

,No Hits (%),No Hits (#),>0 Hits (%),>0 Hits (#)
Baruffaldi,90.319111,7557.0,9.680889,810.0
All Keywords,35.723676,2989.0,64.276324,5378.0
Nakazato & Squicciarini,91.908689,7690.0,8.091311,677.0
Nakazato & Squicciarini + Acronyms,24.046851,2012.0,75.953149,6355.0
Keywords Final,85.801362,7179.0,14.198638,1188.0


## RegEx Search Algorithm

Additionally I use an RegEx search algorithm to check against the results from the Aho-Corasick algorithm.

In [ ]:
# Defining the RegEx search algorithm
def regex_search_keywords(text_data, keywords):
    """
    Searches for AI-related keywords in the given text data using RegEx patterns.

    Parameters:
    - text_data: A pandas Series containing text to search through.
    - keywords: A set of keywords to build RegEx patterns with.

    Returns:
    - A DataFrame with columns for keyword count and matched keywords for each text entry.
    """
    # Pre-compiling RegEx patterns for each keyword to improve performance
    patterns = {keyword: re.compile(r'\b' + re.escape(keyword) + r'\b', re.IGNORECASE) for keyword in keywords}

    # Defining the search function
    def search(text):
        count = 0
        matched_keywords = set()
        for keyword, pattern in patterns.items():
            if pattern.search(text):
                count += 1
                matched_keywords.add(keyword)
        return count, ', '.join(matched_keywords)

    # Applying the search function to the text data
    result = text_data.apply(search)
    
    # Splitting the result into two separate columns
    keyword_count = result.apply(lambda x: x[0])
    matched_keywords = result.apply(lambda x: x[1])
    
    return pd.DataFrame({'regex_keyword_count': keyword_count, 'matched_keywords_regex': matched_keywords})

In [ ]:
data_final_regex = data_final.copy()

# Applying the RegEx search function to the 'text' column with the refined keywords set
regex_results = regex_search_keywords(data_final['text'], keywords_final)

# Adding the results to the 'data' DataFrame
data_final_regex['ai_keyword_count'] = regex_results['regex_keyword_count']
data_final_regex['matched_keywords'] = regex_results['matched_keywords_regex']

# Ensuring the 'regex_keyword_count' is used by the calculate_keyword_distribution function
distribution_regex = calculate_keyword_distribution(data_final_regex)

# Printing the keyword distribution for the RegEx search results
print(distribution_regex)

# Adding the RegEx results to the Overview
distribution_data['RegEx Final'] = distribution_regex
aggregated_df = pd.DataFrame(distribution_data).transpose()
display(aggregated_df)


{'No Hits (%)': 86.87701685191826, 'No Hits (#)': 7269, '>0 Hits (%)': 13.12298314808175, '>0 Hits (#)': 1098}


,No Hits (%),No Hits (#),>0 Hits (%),>0 Hits (#)
Baruffaldi,90.319111,7557.0,9.680889,810.0
All Keywords,35.723676,2989.0,64.276324,5378.0
Nakazato & Squicciarini,91.908689,7690.0,8.091311,677.0
Nakazato & Squicciarini + Acronyms,24.046851,2012.0,75.953149,6355.0
Keywords Final,85.801362,7179.0,14.198638,1188.0
RegEx Final,86.877017,7269.0,13.122983,1098.0


## Comparing the differences between the search algorithms

In [ ]:
data_final_regex.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8367 entries, 0 to 1204
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   website                8367 non-null   object 
 1   text_len               8367 non-null   int64  
 2   source                 8367 non-null   object 
 3   text                   8367 non-null   object 
 4   type                   8367 non-null   object 
 5   closest_snapshot       6596 non-null   object 
 6   closest_snapshot_time  6596 non-null   float64
 7   snapshot_in_window     8367 non-null   float64
 8   text_preprocessed      8367 non-null   object 
 9   ai_keyword_count       8367 non-null   int64  
 10  matched_keywords       8367 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 784.4+ KB


In [ ]:
# Merging the DataFrames on index, expecting automatic suffixes for overlapping column names
# Selecting only texts with keywords
data_final_comp_aho = data_final[data_final['ai_keyword_count'] > 0]
data_final_regex_comp = data_final_regex[data_final_regex['ai_keyword_count']>0]

# Merging the dataframes
data_final_comp = pd.merge(data_final_comp_aho[['text', 'ai_keyword_count','matched_keywords']],
                           data_final_regex_comp[['text', 'ai_keyword_count','matched_keywords']],
                           how = 'outer',
                           left_index = True, right_index=True, 
                           suffixes = ('_aho', '_regex'))

# Saving the data for manual anlysis of the accuracy
data_final_comp.to_excel('Documentation/data_final_comparison.xlsx')

After exporting the merged dataset, I manually checked the differences between the two search algorithms. I found that the Aho-Corasick algorithm has found every ai-related keyword that the RegEx also identifies, and 84 more. Out of these 79 classifications were correct. 

Therefore, I will use the Aho-Corasick search algorithm for the final dataset. 

In [ ]:
# Exporting the search results of the Aho-Corasick algorithm
data_final.to_excel('Documentation/Final Search Results.xlsx')